Import all modules you'll need. if you get an error like : no module "module_name" found; just run "pip install module_name" in empty cell.

In [1]:
%matplotlib Qt
%load_ext autoreload 
# to automaticaly reload all the modules that have been modified
%autoreload 2 
import os
from glob import glob
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, ShuffleSplit
from sklearn import svm, kernel_ridge, metrics, preprocessing
from PIL import Image, ExifTags, ImageFile
import visualize as vis
import preprocessing as pp
import processing as proc
import calculate as cc
import correction as corr
from read_WDF import read_WDF, get_exif
import warnings
warnings.filterwarnings('ignore')

### Reading

In os.chdir, put the enplacement of your .wdf file and run the cell to get all your wdf files.

In [2]:
os.chdir('C:/Users/raoul.missodey/Desktop/Code_python')
files = glob('*.wdf')
for i, j in enumerate(files):
    print(i, ') ', j)

0 )  10nA_5h_obj10.wdf
1 )  800C_355nm.wdf
2 )  bida-a_pt1.wdf
3 )  bida-a_pt2.wdf
4 )  bida-a_pt3.wdf
5 )  bida-a_pt4.wdf
6 )  bida-a_pt5.wdf
7 )  B_carto_rayon_633nm_50%.wdf
8 )  Ca675A325_514nm.wdf
9 )  carto_10238_20x_HD1.wdf
10 )  carto_10238_20x_tr.wdf
11 )  carto_10291.wdf
12 )  carto_50x-bacteries-stramline.wdf
13 )  Carto_plaque_M3C0_7x7cm.wdf
14 )  cit-ACP1_chauffe_25-400°C_488nm_1-34sp.wdf
15 )  depth_Raoul.wdf
16 )  line.wdf
17 )  mapping.wdf
18 )  nax31_C_zone3_carto100x.wdf
19 )  ref-big-carto_633nm-50�-1_CCR.wdf
20 )  ref-big-carto_633nm-50�-1_CCR_Copy1.wdf
21 )  silice_LR.wdf
22 )  Spectre_M3C0_514nm.wdf
23 )  Streamline.wdf
24 )  test.wdf
25 )  UO2 vierge LD2600 633 VV_point.wdf
26 )  UO2 vierge LD2600 633 VV_point_1st_try.wdf
27 )  UO2 vierge LD2600 633 VV_point_Copy1.wdf
28 )  UO2 vierge LD2600 633 VV_point_Copy2nd.wdf
29 )  UO2 vierge LD2600 633 VV_point_grande carte.wdf
30 )  UO2 vierge LD2600 633 VV_streamline.wdf
31 )  UO2 vierge LD2600 633 VV_streamspot.wdf
32 )

In [3]:
filename = files[33] # choose the number of .wdf file you want to use.
# load the file
da, img = read_WDF(filename, verbose=0) # verbose = 1 or true to have all information about your file.

<xarray.DataArray (points: 7921, RamanShifts: 393)>
array([[684.0734 , 667.896  , 642.59595, ..., 492.60477, 524.8775 ,
        482.87344],
       [584.6979 , 605.49744, 587.12006, ..., 415.86734, 406.0373 ,
        430.8717 ],
       [605.49744, 628.60803, 591.74304, ..., 381.21173, 396.13397,
        344.2021 ],
       ...,
       [446.03436, 462.21176, 436.87277, ..., 250.01549, 259.96292,
        264.96133],
       [446.03436, 483.0113 , 427.6268 , ..., 294.5727 , 289.67294,
        264.96133],
       [485.32236, 506.1219 , 499.28317, ..., 279.7203 , 302.05215,
        264.96133]], dtype=float32)
Coordinates:
    X        (points) float64 35.0 35.4 35.8 36.2 36.6 ... 69.0 69.4 69.8 70.2
    Y        (points) float64 10.0 10.0 10.0 10.0 10.0 ... 45.2 45.2 45.2 45.2
    Time     (points) datetime64[ns] 2015-12-02T17:00:51 ... 2015-12-03T04:38:18
    shifts   (RamanShifts) float32 344.8 345.9 347.0 348.1 ... 752.3 753.3 754.3
Dimensions without coordinates: points, RamanShifts
Attributes:
    WdfFlag:             WdfXYXY
    PointsPerSpectrum:   393
    Capacity:            7921
    Count:               7921
    AccumulationCount:   0
    YlistLength:         1
    XlistLength:         393
    DataOriginCount:     5
    ApplicationName:     WiRE
    ApplicationVersion:  3.0.0 build 0
    ScanType:            Static
    MeasurementType:     Map
    StartTime:           2015-12-02 18:00:46
    EndTime:             2015-12-03 05:38:19
    SpectralUnits:       Counts
    LaserWaveLength:     633.0
    Title:               Simple mapping measurement 11
    XlistDataType:       Frequency
    XlistDataUnits:      RamanShift
    YlistDataType:       SpatialY
    YlistDataUnits:      Pixels
    MapAreaType:         InvertedRows
    InitialCoordinates:  [35.   10.   -8.28]
    StepSizes:           [0.4 0.4 1. ]
    NbSteps:             [89 89  1]
    LineFocusSize:       0
    FileSize:            12.3MB
    ScanShape:           (89, 89)
    ColCoord:            X
    RowCoord:            Y

In [9]:
da.data.shape

(7921, 393)

### Visualisation & Preprocessing

In [5]:
vis.show_grid(da, img) # show your Cartography and the scanning part

In [6]:
# show all your spectra and scroll trough them using the slide bar
#you can also enter a number of specific spectra 
show_all_spectra = vis.ShowSpectra(da); 

In [28]:
# after seing all your spectra, if you want to truncate run this cell. you can also add right values.
# if not, pass.
da = pp.select_zone(da, left=200)
after_cutting_check = vis.ShowSpectra(da);

In [10]:
# show all the intensity map depending on wavenumber.
show_all_map = vis.AllMaps(da);

#### Baseline Substraction

In [11]:
#  substract baseline : you have to modify lambda_value(smoothing parameter), p_value(the penalizing weighting factor)
# and lambda1_value(the smoothing parameter for the first derivative of the residual).
check_bline = vis.FindBaseline(da,sigma=da.shifts.data)

In [12]:
# run to see chosen values
print(f"p: {check_bline.p_val:.2e}, lam: {check_bline.lam_val:.2e},lam1: {check_bline.lam1_val:.2e} ")

p: 2.20e-02, lam: 1.09e+06,lam1: 1.00e-03 


In [13]:
# calculate the basline for all spectrum
baseline = cc.baseline_ials(da, p=check_bline.p_val, lam=check_bline.lam_val, lam1= check_bline.lam1_val)

In [14]:
# substraction
db = da.copy()
db.values = db.values - baseline

In [15]:
# show result
after_baseline = vis.ShowSpectra(db)

#### Cosmic Ray Removal

In [16]:
# window_size : elements within shape [1,window_size,1] will get passed to the filter function.
# you can also change the mode{'reflect', 'constant', 'nearest', 'mirror', 'wrap'}. I use mirror by default
# if you see that the algorithm detect a spike but it's not really a spike, you can put the number of this spectrum in not_spike
# and run the cell again
dm = pp.remove_outliers(db,window_size = 5,visualize=1,not_spike=[])

54


In [143]:
vis.ShowSpectra(dm)

#### RPCA (Robust PCA)

In [17]:
# run this cell to see what RPCA does.
# choose noise image and replace it by noise2. if you have .png image, you don't need to divide it by 255 
im = plt.imread("C:/Users/raoul.missodey/Desktop/Code_python/" + "noise5.png")
if im.ndim == 2 : 
    imgray = im.copy()
else :
    imgray = np.mean(im,-1)
A , E = pp.ialm_rpca(imgray)

fig,ax = plt.subplots(nrows = 1,ncols = 3,figsize=(15,8))
ax[0].imshow(imgray,cmap='gray')
ax[0].set_title('X')
ax[1].imshow(A,cmap='gray')
ax[1].set_title('A')
ax[2].imshow(E,cmap='gray')
ax[2].set_title('E')

Text(0.5, 1.0, 'E')

In [26]:
# let's try it on our data.
A, E = pp.ialm_rpca(da.data)
drpca = db.copy()
drpca.values = A
showrpca_spectra = vis.ShowSpectra(drpca) 

In [30]:
# show all maps after rpca
showrpca_map = vis.AllMaps(drpca)

In [62]:
showrpca_spectra = vis.ShowSpectra(db) 

In [29]:
drpca1 = db.copy()
drpca1.values = E
showrpca_spectra = vis.ShowSpectra(drpca1) 
showrpca_map = vis.AllMaps(drpca1)

#### Normalization

In [18]:
# choose normalize method you want to apply : min_max, l1, l2, max, area, robust_scale 
#dm.values -= np.min(dm.values, axis=-1, keepdims=True)
dn = pp.normalize(dm, method="min_max")
after_normalize = vis.ShowSpectra(dn)

### Processing

#### PCA

In [28]:
# compute pca using svd decomposition
dsvd ,var,scores,loadings = pp.svd_decomp(dn, n_components=11, visualize_clean=1, visualize_components=1,visualize_err=1)

In [21]:
# in var we have "explained variance" of each components
var

array([98.1,  1.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ])

In [23]:
# choose number of component you want to keep
dsvd ,var,score ,loadings = pp.svd_decomp(dn, n_components=7, visualize_clean=1, visualize_components=1,visualize_err=1)

#### NMF Decomposition

In [24]:
# Run this cell to see pure spectrum and pure contribution
dnm = proc.deconvolve_nmf(dn, n_components=10, visualize_components=1, visualize_compare=1, visualize_err=1)

Error increased above fractionalctol_increase (C iter). Exiting


#### HCA (clustering)

In [25]:
# run this cell will show the dendrogramm :
# you can change the method you want to use to calculate the distance between your classes : average, single, complete,ward...
# also you can change the metric you want to use to calculate the distance between your spectrum : 
#'chebyshev', 'cityblock', 'correlation', 'cosine', 'dice', 'euclidean'...
dhca,spectra_hca = proc.hca(dsvd,method='ward',metric ='euclidean',visualize=1)

In [27]:
# after you have visualized your dendrogramm choose where you want to cute it : (t)
# then replace t by this value and run the cell to visualize the result
# criterion : inconsistent, distance, maxclust...
#right click on the map to see the corresponding spectrum in cluster
showhca = vis.ShowHca(dhca, dsvd,spectra_hca,t=40,criterion='distance')

#### KMeans (cluster analysis in WiRE)

In [86]:
# run this cell to see inertia curve. you can also changed the maximalumber of your cluster ; warn : big number can make
# the algorithm slow
inertia = vis.ShowElbpt(30, dsvd)

In [87]:
# see the number of your maximal cluster given by Elbow method , warn : use the same maximal number of cluster
# curve : convex or concave depending on the look of your curve
# direction : decreasing or increasing
vis.print_elbowpt(30, inertia, curve="convex", direction="decreasing")

4

In [88]:
# enter the number of cluster given by the Elbow method
# again right click on the map to see the corresponding spectrum in cluster
showkmeans = vis.ShowKMeans(dsvd,nb_clas= 4) 